In [ ]:
# Sentiment vs Trader Performance

In [1]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from scipy import stats
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, roc_curve

In [2]:
# Plotting defaults
plt.rcParams['figure.figsize'] = (8,4)

In [3]:
SENT_FILE = 'fear_greed_index.csv' 
TRADES_FILE = 'historical_data.csv'
OUT_DIR = Path('analysis_outputs')
OUT_DIR.mkdir(exist_ok=True)

In [4]:
def safe_read(path):
    path = Path(path)
    if not path.exists():
        raise FileNotFoundError(f"File not found: {path}")
    
    # Try common formats
    if path.suffix.lower() in ['.csv', '.txt']:
        return pd.read_csv(path, low_memory=False)
    if path.suffix.lower() in ['.parquet', '.pq']:
        return pd.read_parquet(path)
    
    # Fallback
    return pd.read_csv(path, low_memory=False)

In [5]:
def find_column_like(df, keywords):
    keys = [k.lower() for k in keywords]
    for c in df.columns:
        low = c.lower()
        for k in keys:
            if k in low:
                return c
    return None

In [6]:
sent = safe_read(SENT_FILE)
trades = safe_read(TRADES_FILE)


print('Sentiment rows, cols:', sent.shape)
print('Trades rows, cols:', trades.shape)

Sentiment rows, cols: (2644, 4)
Trades rows, cols: (211224, 16)


In [7]:
print('\nSentiment head:')
print(sent.head())
print('\nTrades head:')
print(trades.head())


Sentiment head:
    timestamp  value classification        date
0  1517463000     30           Fear  2018-02-01
1  1517549400     15   Extreme Fear  2018-02-02
2  1517635800     40           Fear  2018-02-03
3  1517722200     24   Extreme Fear  2018-02-04
4  1517808600     11   Extreme Fear  2018-02-05

Trades head:
                                      Account  Coin  Execution Price  \
0  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9769   
1  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9800   
2  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9855   
3  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9874   
4  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9894   

   Size Tokens  Size USD Side     Timestamp IST  Start Position Direction  \
0       986.87   7872.16  BUY  02-12-2024 22:50        0.000000       Buy   
1        16.00    127.68  BUY  02-12-2024 22:50      986.524596       Buy   
2       144.09   

In [8]:
# Inspect columns and detect key fields
# - We want: date column in sentiment, classification column in sentiment,
# timestamp column in trades, and closed PnL (or realized PnL) in trades.

In [9]:
sent_date_col = find_column_like(sent, ['date','time','timestamp','day'])
sent_label_col = find_column_like(sent, ['class','sentiment','label','value','fear','greed'])
trades_time_col = find_column_like(trades, ['time','timestamp','date','datetime'])
trades_pnl_col = find_column_like(trades, ['pnl','closed','realized','profit','loss'])

In [10]:
print('Detected sentiment date column:', sent_date_col)
print('Detected sentiment label column:', sent_label_col)
print('Detected trades time column:', trades_time_col)
print('Detected trades pnl column:', trades_pnl_col)

Detected sentiment date column: timestamp
Detected sentiment label column: value
Detected trades time column: Timestamp IST
Detected trades pnl column: Closed PnL


In [12]:
# parse sentiment date
if sent_date_col is None:
    sent['Date_parsed'] = pd.to_datetime(sent.iloc[:,0], errors='coerce')
else:
    sent['Date_parsed'] = pd.to_datetime(sent[sent_date_col], errors='coerce', utc=True)


sent['date_only'] = sent['Date_parsed'].dt.date

In [13]:
# parse trades time
if trades_time_col is None:
    trades['Time_parsed'] = pd.to_datetime(trades.iloc[:,0], errors='coerce')
else:
    trades['Time_parsed'] = pd.to_datetime(trades[trades_time_col], errors='coerce', utc=True)


trades['trade_date'] = trades['Time_parsed'].dt.date


print('Parsed sample sentiment dates:')
print(sent[['Date_parsed','date_only']].head())
print('\nParsed sample trade dates:')
print(trades[['Time_parsed','trade_date']].head())

Parsed sample sentiment dates:
                          Date_parsed   date_only
0    1970-01-01 00:00:01.517463+00:00  1970-01-01
1 1970-01-01 00:00:01.517549400+00:00  1970-01-01
2 1970-01-01 00:00:01.517635800+00:00  1970-01-01
3 1970-01-01 00:00:01.517722200+00:00  1970-01-01
4 1970-01-01 00:00:01.517808600+00:00  1970-01-01

Parsed sample trade dates:
                Time_parsed  trade_date
0 2024-02-12 22:50:00+00:00  2024-02-12
1 2024-02-12 22:50:00+00:00  2024-02-12
2 2024-02-12 22:50:00+00:00  2024-02-12
3 2024-02-12 22:50:00+00:00  2024-02-12
4 2024-02-12 22:50:00+00:00  2024-02-12


In [14]:
# Normalize sentiment labels
# - Turn values like 'fear/greed index' into category labels: Fear, Greed, Neutral
# - If your sentiment file already has a classification column (like 'classification'), use it.

In [15]:
if sent_label_col is None:
    # choose a fallback (non-date column)
    possible = [c for c in sent.columns if c not in (sent_date_col, 'Date_parsed','date_only')]
    sent_label_col = possible[0]


sent['sent_class'] = sent[sent_label_col].astype(str).str.strip().str.title()
print('Sentiment class sample:')
print(sent[['Date_parsed', sent_label_col, 'sent_class']].head())

Sentiment class sample:
                          Date_parsed  value sent_class
0    1970-01-01 00:00:01.517463+00:00     30         30
1 1970-01-01 00:00:01.517549400+00:00     15         15
2 1970-01-01 00:00:01.517635800+00:00     40         40
3 1970-01-01 00:00:01.517722200+00:00     24         24
4 1970-01-01 00:00:01.517808600+00:00     11         11
